# Install, Paths and Parameters

In [ ]:
# This extension reloads external Python files
import os
from pathlib import Path
import getpass
import numpy as np
import pandas as pd
import time
import math
from collections import defaultdict

import torch
from torch.utils.data import DataLoader
from torch import nn
from tqdm import tqdm
import random
import sys
from torch.utils.data import random_split
from matplotlib import pyplot as plt

# allow imports when running script from within project dir
[sys.path.append(i) for i in ['.', '..']]

# local
from src.helpers.helpers import get_random_indexes, get_random_classes
from src.model.dino_model import get_dino
from src.model.train import *
from src.model.data import *

# seed
SEED = 42
random.seed(SEED)
torch.manual_seed(SEED)
np.random.seed(SEED)

username = getpass.getuser()
DATA_PATH = Path('/','cluster', 'scratch', 'thobauma', 'dl_data')
MAX_PATH = Path('/','cluster', 'scratch', 'mmathys', 'dl_data')
# Path for intermediate outputs
BASE_POSTHOC_PATH = Path(MAX_PATH, 'posthoc-fixed-labels/')
#BASE_POSTHOC_PATH = Path(MAX_PATH, 'posthoc-subset/')

# Original Dataset
ORI_PATH = Path(DATA_PATH, 'ori_data/')
CLASS_SUBSET_PATH = Path(ORI_PATH, 'class_subset.npy')

TR_PATH = Path(ORI_PATH, 'train/')
TR_ORI_LABEL_PATH = Path(TR_PATH,'correct_labels.txt')
TR_ORI_IMAGES_PATH = Path(TR_PATH,'images')

VAL_PATH = Path(ORI_PATH, 'validation/')
VAL_ORI_LABEL_PATH = Path(VAL_PATH,'correct_labels.txt')
VAL_ORI_IMAGES_PATH = Path(VAL_PATH,'images')

# DAmageNet
DN_PATH = Path(DATA_PATH, 'damageNet')
DN_LABEL_PATH = Path(DN_PATH, 'val_damagenet.txt')
DN_IMAGES_PATH = Path(DN_PATH, 'images')
DN_POSTHOC_PATH = Path(BASE_POSTHOC_PATH, 'damagenet')
DN_POSTHOC_LABEL_PATH = Path(DN_POSTHOC_PATH, 'labels.csv')

# PGD
TR_PGD_PATH = Path(MAX_PATH, 'adversarial_data/pgd_06/train')
TR_PGD_LABEL_PATH = TR_ORI_LABEL_PATH
TR_PGD_IMAGES_PATH = Path(TR_PGD_PATH, 'images')
TR_PGD_POSTHOC_PATH = Path(BASE_POSTHOC_PATH, 'pgd/train/')
TR_PGD_POSTHOC_LABEL_PATH = Path(TR_PGD_POSTHOC_PATH, 'labels.csv')

VAL_PGD_PATH = Path(MAX_PATH, 'adversarial_data/pgd_06/validation')
VAL_PGD_LABEL_PATH = VAL_ORI_LABEL_PATH
VAL_PGD_IMAGES_PATH = Path(VAL_PGD_PATH, 'images')
VAL_PGD_POSTHOC_PATH = Path(BASE_POSTHOC_PATH, 'pgd/validation/')
VAL_PGD_POSTHOC_LABEL_PATH = Path(VAL_PGD_POSTHOC_PATH, 'labels.csv')

# CW
TR_CW_PATH = Path(MAX_PATH, 'adversarial_data/cw/train')
TR_CW_LABEL_PATH = TR_ORI_LABEL_PATH
TR_CW_IMAGES_PATH = Path(TR_CW_PATH, 'images')
TR_CW_POSTHOC_PATH = Path(BASE_POSTHOC_PATH, 'cw/train/')
TR_CW_POSTHOC_LABEL_PATH = Path(TR_CW_POSTHOC_PATH, 'labels.csv')

VAL_CW_PATH = Path(MAX_PATH, 'adversarial_data/cw/validation')
VAL_CW_LABEL_PATH = VAL_ORI_LABEL_PATH
VAL_CW_IMAGES_PATH = Path(VAL_CW_PATH, 'images')
VAL_CW_POSTHOC_PATH = Path(BASE_POSTHOC_PATH, 'cw/validation/')
VAL_CW_POSTHOC_LABEL_PATH = Path(VAL_CW_POSTHOC_PATH, 'labels.csv')

# FGSM
TR_FGSM_PATH = Path(MAX_PATH, 'adversarial_data/fgsm_06/train')
TR_FGSM_LABEL_PATH = TR_ORI_LABEL_PATH
TR_FGSM_IMAGES_PATH = Path(TR_FGSM_PATH, 'images')
TR_FGSM_POSTHOC_PATH = Path(BASE_POSTHOC_PATH, 'fgsm/train/')
TR_FGSM_POSTHOC_LABEL_PATH = Path(TR_FGSM_POSTHOC_PATH, 'labels.csv')

VAL_FGSM_PATH = Path(MAX_PATH, 'adversarial_data/fgsm_06/validation')
VAL_FGSM_LABEL_PATH = VAL_ORI_LABEL_PATH
VAL_FGSM_IMAGES_PATH = Path(VAL_FGSM_PATH, 'images')
VAL_FGSM_POSTHOC_PATH = Path(BASE_POSTHOC_PATH, 'fgsm/validation/')
VAL_FGSM_POSTHOC_LABEL_PATH = Path(VAL_FGSM_POSTHOC_PATH, 'labels.csv')

In [ ]:
# If CLASS_SUBSET is specified, INDEX_SUBSET will be ignored. Set CLASS_SUBSET=None if you want to use indexes.
# INDEX_SUBSET = get_random_indexes(number_of_images = 50000, n_samples=1000)
# CLASS_SUBSET = get_random_classes(number_of_classes = 25, min_rand_class = 1, max_rand_class = 1001)


CLASS_SUBSET = np.load(CLASS_SUBSET_PATH)
# CLASS_SUBSET = CLASS_SUBSET[:2]

INDEX_SUBSET = None
NUM_WORKERS= 0
PIN_MEMORY=True

BATCH_SIZE = 64

DEVICE = 'cuda'

In [ ]:
datasets_paths = {
            'cw':{ 
                'b':{
                    'train':{
                        'label':TR_ORI_LABEL_PATH,
                        'images':TR_CW_IMAGES_PATH
                    },
                    'val':
                    {
                        'label':VAL_ORI_LABEL_PATH,
                        'images':VAL_CW_IMAGES_PATH
                    }
                }
            },
            'ori':{
                'b':{
                    'train':{
                        'label':TR_ORI_LABEL_PATH,
                        'images':TR_ORI_IMAGES_PATH
                    },
                    'val':{
                        'label':VAL_ORI_LABEL_PATH,
                        'images':VAL_ORI_IMAGES_PATH
                    }
                }
            },
            'dn':{
                'b':{
                    'train':{
                        'label':TR_CW_PATH,
                        'images':None
                    },
                    'val':
                    {
                        'label':VAL_ORI_LABEL_PATH,
                        'images':DN_IMAGES_PATH
                    }
                 }
            },
            'fgsm_06':{
                'b':{
                    'train':{
                        'label':TR_ORI_LABEL_PATH,
                        'images':TR_FGSM_IMAGES_PATH
                    },
                    'val':
                    {
                        'label':VAL_ORI_LABEL_PATH,
                        'images':VAL_FGSM_IMAGES_PATH
                    }
                 }
            },
            'pgd_06':{
                'b':{
                    'train':{
                        'label':TR_ORI_LABEL_PATH,
                        'images':TR_PGD_IMAGES_PATH
                    },
                    'val':
                    {
                        'label':VAL_ORI_LABEL_PATH,
                        'images':VAL_PGD_IMAGES_PATH
                    }
                }
            }
}

datasets = ['ori', 'cw', 'pgd_06', 'fgsm_06']
for ds in datasets:
    ds_dict = datasets_paths[ds]
    ds_dict['p'] = {
        'train': { 
            'images': Path(BASE_POSTHOC_PATH, ds, 'train', 'images'),
            'label': Path(BASE_POSTHOC_PATH, ds, 'train', 'labels.csv')
        },
        'val': { 
            'images': Path(BASE_POSTHOC_PATH, ds, 'val', 'images'),
            'label': Path(BASE_POSTHOC_PATH, ds, 'val', 'labels.csv')
        }
    }
    

In [ ]:
datasets_paths['cw']

# Import DINO
Official repo: https://github.com/facebookresearch/dino

In [ ]:
class LinearClassifier(nn.Module):
    """Linear layer to train on top of frozen features"""
    def __init__(self, dim, num_labels=1000):
        super(LinearClassifier, self).__init__()
        self.num_labels = num_labels
        self.linear = nn.Linear(dim, num_labels)
        self.linear.weight.data.normal_(mean=0.0, std=0.01)
        self.linear.bias.data.zero_()

    def forward(self, x):
        # flatten
        x = x.view(x.size(0), -1)

        # linear layer
        return self.linear(x)
    
model, dino_classifier = get_dino()

linear_classifier = LinearClassifier(dino_classifier.linear.in_features, 
                         num_labels=len(CLASS_SUBSET))

linear_classifier.load_state_dict(torch.load("/cluster/scratch/mmathys/dl_data/adversarial_data/adv_classifiers/25_classes" + "/" + "clean.pt"))
linear_classifier.cuda()


from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()
label_encoder.fit([i for i in CLASS_SUBSET])

# Forward Pass

In [ ]:
def posthoc_forward_pass(model, classifier, datasets, datasets_paths):
    logger_dict = {}
    for ds in datasets:
        ds_b = datasets_paths[ds]['b']
        ds_p = datasets_paths[ds]['p']
        
        logger_dict[ds] = {}
        
        transform = ONLY_NORMALIZE_TRANSFORM
        
        if ds == 'ori':
            transform = ORIGINAL_TRANSFORM
            
        for tv in ['train', 'val']:            
            print(f'''images: {ds_b[tv]['images']}\nlabel: {ds_b[tv]['label']}\npred: {ds_p[tv]['label']}''')
            
            data_set = AdvTrainingImageDataset(img_folder=ds_b[tv]['images'], 
                                               labels_file_name=ds_b[tv]['label'], 
                                               #labels_file_name='/cluster/home/thobauma/deeplearning/data-mmathys/adversarial_data_test/cw/train/labels.txt',
                                               transform=transform, 
                                               class_subset=CLASS_SUBSET,
                                               index_subset=None,
                                               label_encoder=label_encoder)
            
            data_loader = DataLoader(data_set, 
                                     batch_size=BATCH_SIZE, 
                                     num_workers=NUM_WORKERS, 
                                     pin_memory=PIN_MEMORY, 
                                     shuffle=False)
            
            print(f'''{ds}: {tv} {len(data_set)}''')
            logger_dict[ds][tv] = validate_network(model=model, 
                                                   classifier=classifier, 
                                                   validation_loader=data_loader, 
                                                   criterion=nn.CrossEntropyLoss(), 
                                                   tensor_dir=ds_p[tv]['images'],
                                                   adversarial_attack=None, 
                                                   n=4, 
                                                   avgpool_patchtokens=False, 
                                                   path_predictions=ds_p[tv]['label'],
                                                   show_image=False)
            
    return logger_dict

In [ ]:
data_sets = ['ori', 'pgd_06', 'fgsm_06', 'cw']
logger_dict = posthoc_forward_pass(model,
                                   linear_classifier, 
                                   data_sets, 
                                   datasets_paths)

# Create Labels

In [ ]:
from functools import reduce

def merge_frames(frames, on_what=['file', 'true_labels'], how='left'):
    merged_df = reduce(lambda left, right:pd.merge(left, right, on=on_what, how=how,  suffixes=('', '_drop')), frames)
    merged_df.drop(merged_df.filter(regex='_drop$').columns.tolist(),axis=1, inplace=True)
    return merged_df

def get_merged_labels(datasets=['ori', 'cw', 'pgd_06', 'fgsm_06'], datasets_types=['train', 'val'], datasets_paths=datasets_paths, save_path=BASE_POSTHOC_PATH, get_df_dict = False):
    df_data_types = {}
    df_data = {}
    for tv in datasets_types:
        df_data[tv] = {}
        for ds in datasets:
            ds_dict = datasets_paths[ds]
            df_data[tv][ds] = pd.read_csv(ds_dict['p'][tv]['label'])
            df_data[tv][ds].rename(columns = {'pred_labels': ds+'_pred'}, inplace = True)
            if ds != 'ori':
                df_data[tv][ds] = pd.merge(df_data[tv][ds], df_data[tv]['ori'], on=['file', 'true_labels'], how='left')
            df_data[tv][ds].to_csv(Path(BASE_POSTHOC_PATH, ds, tv, 'labels_merged.csv'), sep=",", index=None)
        df_data_types[tv] = merge_frames(df_data[tv].values())
        if save_path is not None:
            df_data_types[tv].to_csv(Path(save_path,tv+'.csv'), sep=",", index=None)
    if get_df_dict:
        return df_data_types, df_data

    return df_data_types

    

In [ ]:
df_types, df_data = get_merged_labels(get_df_dict=True)

In [ ]:
df_types['train']

In [ ]:
df_data['train']['cw']

In [ ]:
name = 'cw'
df = df_data['train'][name]

In [ ]:
for name in ['cw', 'pgd_06', 'fgsm_06']:    
    df = df_data['train'][name]
    print(f'''{name}''')
    ldf = len(df)
    print(f'''total data:             {ldf}''')
    print(f'''correct pred:           {len(df[df['true_labels']==df['ori_pred']])},   {len(df[df['true_labels']==df['ori_pred']])/ldf}''')
    print(f'''incorrect adv pred:     {len(df[df['true_labels']!=df[name+'_pred']])},   {len(df[df['true_labels']!=df[name+'_pred']])/ldf}''')
    df_f = df[df['true_labels']==df['ori_pred']]
    print(f'''number adv tuples:      {len(df_f[df_f['true_labels']!=df_f[name+'_pred']])},   {len(df_f[df_f['true_labels']!=df_f[name+'_pred']])/ldf}''')
    print(f'''\n''')

In [ ]:
cw_df = pd.read_csv('/cluster/home/thobauma/deeplearning/data-mmathys/adversarial_data_test/cw/train/labels.txt', sep=" ", header=None)
cw_df.columns=['file', 'label']

In [ ]:
cw_df

In [ ]:
cw_stuff = pd.merge(cw_df, df_data['train']['cw'], how='left', suffixes=('', '_post'), on=['file'])

In [ ]:
cw_stuff